# Analyze results
Aug 28, 2020

In [1]:
import     numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize
import seaborn as sns

In [2]:
from ipywidgets import *

In [3]:
%matplotlib widget

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/modules_image_analysis/')
from modules_img_analysis import *

In [5]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/code/1_basic_GAN/1_main_code/')
import post_analysis_pandas as post


In [6]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4.) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s)


## Read data

In [7]:
# main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/'
# results_dir=main_dir+'20201002_064327'

In [8]:
img_size=128
dict1={'128':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/',
      '512':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/512sq/'}

u=interactive(lambda x: dict1[x], x=Select(options=dict1.keys()))
# display(u)


In [9]:
# parent_dir=u.result
parent_dir=dict1[str(img_size)]

dir_lst=[i.split('/')[-1] for i in glob.glob(parent_dir+'20*')]
w=interactive(lambda x: x, x=Dropdown(options=dir_lst))
display(w)

interactive(children=(Dropdown(description='x', options=('20210305_085747_lambda2.0', '20210305_085850_lambda2…

In [10]:
# result_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/simple_gan_best_runs/20201213_173446_lambda0.1_good_models'

In [11]:
result=w.result
result_dir=parent_dir+result
print(result_dir)

/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/pytorch/results/128sq/20210401_080556_train


### Get validation data

In [12]:
# Get location for validation data
raw_data_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/'
input_files_dict={'128':raw_data_dir+'128_square/dataset_2_smoothing_200k/norm_1_train_val.npy',
                 '512':raw_data_dir+'512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'}

In [13]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_sdev=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_sdev':spec_sdev }
    return dict1

### Extract validation data
num_bkgnd=1000
fname=input_files_dict[str(img_size)]
s_val=np.load(fname,mmap_mode='r')[:num_bkgnd][:,0,:,:]
print(s_val.shape)

bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1) 
### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)

(1000, 128, 128)


## Plot Losses

In [12]:
df_metrics=pd.read_pickle(result_dir+'/df_metrics.pkle').astype(np.float64)


In [13]:
df_metrics.tail(10)

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
1,1.0,0.0,151.419983,8727.786133,8879.206055,0.0,inf,inf,2.020478,41.803719,2.171167,NaN,0.001998,10207.686523,8814.615234,8814.615234,1.006878
2,2.0,1.0,100.623550,8729.311523,8829.935547,0.0,37.514938,37.512917,1.927323,39.340347,2.024011,NaN,0.002021,10191.889648,8769.349609,8769.349609,1.006923
3,3.0,1.0,150.052628,7750.361816,7900.414551,0.0,33.981091,33.979107,1.955906,33.952698,1.901283,NaN,0.001983,10147.607422,7857.298340,7857.298340,0.996975
4,4.0,2.0,188.771790,8363.617188,8552.388672,0.0,43.767567,43.765617,1.888072,43.145172,1.956569,NaN,0.001951,10138.222656,8434.011719,8434.011719,0.998674
5,5.0,2.0,143.930222,8489.005859,8632.936523,0.0,37.463936,37.462055,1.856824,43.927513,1.839142,NaN,0.001881,10131.258789,8566.894531,8566.894531,0.988187
6,6.0,3.0,145.553314,7855.338867,8000.892090,0.0,33.692234,33.690468,1.881493,34.031216,1.809892,NaN,0.001765,10115.888672,8076.560547,8076.560547,1.024691
7,7.0,3.0,155.816605,7794.276367,7950.092773,0.0,42.821873,42.820190,1.876664,45.513916,1.842845,NaN,0.001682,10076.692383,8064.732422,8064.732422,1.201532
8,8.0,4.0,160.035172,7843.662109,8003.697266,0.0,43.740566,43.738972,1.892394,42.609337,1.822600,NaN,0.001595,10084.660156,7902.696289,7902.696289,1.063119
9,9.0,4.0,132.025299,7560.716309,7692.741699,0.0,38.579342,38.577789,1.900531,34.186943,1.855985,NaN,0.001553,10051.861328,7710.831055,7710.831055,1.006335
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.848042,1.823254,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
def f_plot_metrics(df,col_list):
    
    plt.figure()
    for key in col_list:
        plt.plot(df_metrics[key],label=key,marker='*',linestyle='')
    plt.legend()
    
#     col_list=list(col_list)
#     df.plot(kind='line',x='step',y=col_list)
    
# f_plot_metrics(df_metrics,['spec_chi','hist_chi'])

interact_manual(f_plot_metrics,df=fixed(df_metrics), col_list=SelectMultiple(options=df_metrics.columns.values))

interactive(children=(SelectMultiple(description='col_list', options=('step', 'epoch', 'Dreal', 'Dfake', 'Dful…

<function __main__.f_plot_metrics(df, col_list)>

In [17]:

chi=df_metrics.quantile(q=0.005,axis=0)['hist_chi']
print(chi)
df_metrics[df_metrics['hist_chi']<=chi].sort_values(by=['hist_chi']).head(10)

-1.6125873601436616


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
9165,9165.0,5.0,0.126403,0.578895,0.705298,0.830199,9.205657,8.375458,-3.371428,8.520576,-3.626162,0.0,0.0,4.321398,-0.253434,-0.253434,0.322318
9164,9164.0,5.0,0.265508,12.484107,12.749615,0.000003,8.511552,8.511549,-3.225687,8.487927,-3.340391,0.0,0.0,15.894117,12.684171,12.684171,0.321636
9166,9166.0,5.0,0.256582,12.641390,12.897972,0.000003,8.556256,8.556253,-3.062916,8.237353,-3.305876,0.0,0.0,16.454300,12.840198,12.840198,0.314992
9163,9163.0,5.0,0.163011,0.531486,0.694497,0.903172,9.106766,8.203593,-3.268511,8.234748,-3.241067,0.0,0.0,3.921676,-0.378757,-0.378757,0.321865
9161,9161.0,5.0,0.229774,0.329963,0.559737,1.321376,9.717937,8.396562,-3.212570,8.507963,-3.213125,0.0,0.0,3.369017,-1.009110,-1.009110,0.336941
9160,9160.0,5.0,0.252435,11.749527,12.001962,0.000007,8.520497,8.520491,-3.198966,8.381926,-3.153173,0.0,0.0,15.411587,11.938740,11.938740,0.316522
9162,9162.0,5.0,0.214647,12.293420,12.508067,0.000004,8.642687,8.642683,-3.230917,8.120164,-3.076869,0.0,0.0,15.737246,12.490650,12.490650,0.317677
9179,9179.0,5.0,0.203169,4.992001,5.195169,0.010416,8.257793,8.247377,-3.020860,8.198448,-3.049831,0.0,0.0,12.945306,5.054736,5.054736,0.313001
9159,9159.0,5.0,0.254654,0.182889,0.437543,2.021660,10.117157,8.095497,-3.122787,8.182856,-2.971455,0.0,0.0,2.918025,-1.868428,-1.868428,0.320301
9177,9177.0,5.0,2.938682,0.141518,3.080199,9.589148,17.961582,8.372435,-2.994325,8.573198,-2.935057,0.0,0.0,-1.915371,-9.589024,-9.589024,0.320725


In [18]:
display(df_metrics.sort_values(by=['hist_chi']).head(8))
display(df_metrics.sort_values(by=['spec_chi']).head(8))

,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
9165,9165.0,5.0,0.126403,0.578895,0.705298,0.830199,9.205657,8.375458,-3.371428,8.520576,-3.626162,0.0,0.0,4.321398,-0.253434,-0.253434,0.322318
9164,9164.0,5.0,0.265508,12.484107,12.749615,0.000003,8.511552,8.511549,-3.225687,8.487927,-3.340391,0.0,0.0,15.894117,12.684171,12.684171,0.321636
9166,9166.0,5.0,0.256582,12.641390,12.897972,0.000003,8.556256,8.556253,-3.062916,8.237353,-3.305876,0.0,0.0,16.454300,12.840198,12.840198,0.314992
9163,9163.0,5.0,0.163011,0.531486,0.694497,0.903172,9.106766,8.203593,-3.268511,8.234748,-3.241067,0.0,0.0,3.921676,-0.378757,-0.378757,0.321865
9161,9161.0,5.0,0.229774,0.329963,0.559737,1.321376,9.717937,8.396562,-3.212570,8.507963,-3.213125,0.0,0.0,3.369017,-1.009110,-1.009110,0.336941
9160,9160.0,5.0,0.252435,11.749527,12.001962,0.000007,8.520497,8.520491,-3.198966,8.381926,-3.153173,0.0,0.0,15.411587,11.938740,11.938740,0.316522
9162,9162.0,5.0,0.214647,12.293420,12.508067,0.000004,8.642687,8.642683,-3.230917,8.120164,-3.076869,0.0,0.0,15.737246,12.490650,12.490650,0.317677
9179,9179.0,5.0,0.203169,4.992001,5.195169,0.010416,8.257793,8.247377,-3.020860,8.198448,-3.049831,0.0,0.0,12.945306,5.054736,5.054736,0.313001


,step,epoch,Dreal,Dfake,Dfull,G_adv,G_full,spec_loss,hist_loss,spec_chi,hist_chi,gp_loss,fm_loss,D(x),D_G_z1,D_G_z2,time
25844,25844.0,16.0,0.115365,5.535372,5.650737,0.003978,7.723255,7.719277,0.585232,7.440961,0.498061,0.0,0.0,7.321600,5.621196,5.621196,0.333781
25082,25082.0,16.0,0.175530,0.531724,0.707253,0.960902,8.676497,7.715595,0.492198,7.462867,0.399464,0.0,0.0,2.016663,-0.440652,-0.440652,0.324773
24744,24744.0,15.0,0.354607,0.455986,0.810592,1.093749,9.175434,8.081685,0.861975,7.467211,1.228933,0.0,0.0,1.104684,-0.633469,-0.633469,0.334303
30718,30718.0,19.0,0.309495,0.525896,0.835391,0.941700,9.259689,8.317989,0.955066,7.468265,1.017910,0.0,0.0,1.265532,-0.414745,-0.414745,0.325644
25728,25728.0,16.0,0.115521,5.055433,5.170954,0.006284,7.599692,7.593409,0.869664,7.470405,1.132266,0.0,0.0,7.316003,5.134998,5.134998,0.324926
26752,26752.0,17.0,0.114625,5.407657,5.522282,0.004659,7.853621,7.848962,0.928407,7.472019,0.867247,0.0,0.0,7.546748,5.489364,5.489364,0.327677
18974,18974.0,12.0,0.109274,5.657246,5.766520,0.003466,7.643982,7.640516,1.065898,7.483041,1.141783,0.0,0.0,7.341938,5.749311,5.749311,0.353779
25753,25753.0,16.0,0.279593,0.587146,0.866739,0.845633,8.694573,7.848941,0.726750,7.483757,0.759788,0.0,0.0,1.335957,-0.251625,-0.251625,0.327875


## Read stored chi-squares for images

In [19]:
df=pd.read_pickle(result_dir+'/df_processed.pkle')
df[['epoch','step']]=df[['epoch','step']].astype(int)
df['label']=df.epoch.astype(str)+'-'+df.step.astype(str) # Add label column for plotting

col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']


In [20]:
def f_slice_df(df,cutoff=0.2,sort_col='chi_1',col_mode='all',head=10,epoch_range=[0,None],display_flag=False):
    ''' View dataframe after slicing
    '''
    col_list=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2','epoch','step']
    if (col_mode=='short'): col_list=['label','chi_1','chi_spec1']
        
    print(epoch_range)
    if epoch_range[1]==None: epoch_range[1]=df.max()['epoch']
    df=df[(df.epoch<=epoch_range[1])&(df.epoch>=epoch_range[0])]

    ## Apply cutoff to keep reasonable chi1 and chispec1
    best_keys=[ 'chi_1', 'chi_spec1']
    q_dict=dict(df.quantile(q=cutoff,axis=0)[best_keys])   
    print(q_dict)
    df=df.query('chi_1 < {0} & chi_spec1 < {1}'.format(q_dict['chi_1'],q_dict['chi_spec1']))
    
    # Sort dataframe
    df1=df[df.epoch>0].sort_values(by=sort_col)
        
    df2=df1.head(head)[col_list]

    if display_flag:   # Display df
        display_cols=['label','chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2']
        display(df2)
    
    return df2

In [21]:
w=interactive(f_slice_df,df=fixed(df),
cutoff=widgets.FloatSlider(value=0.2, min=0, max=1.0, step=0.01), 
col_mode=['all','short'], display_flag=widgets.Checkbox(value=False),
head=widgets.IntSlider(value=10,min=1,max=20,step=1),
epoch_range=widgets.IntRangeSlider(value=[0,np.max(df.epoch.values)],min=0,max=np.max(df.epoch.values),step=1),
sort_col=['chi_1','chi_spec1', 'chi_spec3', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2'],
)
display(w)

interactive(children=(FloatSlider(value=0.2, description='cutoff', max=1.0, step=0.01), Dropdown(description='…

In [22]:
df_sliced=w.result
# df_sliced

In [23]:
best_step=[]
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_1',head=4,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.8,sort_col='chi_1c',head=2,display_flag=False).step.values)
best_step.append(f_slice_df(df,cutoff=0.4,sort_col='chi_spec1',head=2,display_flag=False).step.values)
print(best_step)
best_step=np.unique([i for j in best_step for i in j])
print(best_step)

[0, None]
{'chi_1': 0.2450134622971836, 'chi_spec1': 4.318770960029531}
[0, 19]
{'chi_1': 4.490743473542528, 'chi_spec1': 19299.012896028908}
[0, 19]
{'chi_1': 0.2450134622971836, 'chi_spec1': 4.318770960029531}
[array([29420, 28040, 28170, 18170]), array([29770, 17440]), array([25910, 25900])]
[17440 18170 25900 25910 28040 28170 29420 29770]


In [24]:
# best_step=[58800]
# best_step=np.arange(32800,32900,10).astype(int)

In [25]:
df_best=df[df.step.isin(best_step)]
print(df_best.shape)
print([(df_best[df_best.step==step].epoch.values[0],df_best[df_best.step==step].step.values[0]) for step in best_step])
# print([(df_best.loc[idx].epoch,df_best.loc[idx].step) for idx in best_idx])

(8, 22)
[(11, 17440), (11, 18170), (16, 25900), (16, 25910), (17, 28040), (18, 28170), (18, 29420), (19, 29770)]


In [26]:
df_best[col_list]

,label,chi_1,chi_spec1,chi_spec3,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,epoch,step
1743,11-17440,0.171949,3.587175,31.412641,0.164329,0.006865,0.000755,0.128350,10.446924,26.218715,11,17440
1816,11-18170,0.071424,4.202006,35.267789,0.036675,0.029868,0.004880,0.025077,13.742778,35.506343,11,18170
2589,16-25900,0.219114,0.933127,33.651707,0.090704,0.069528,0.058882,0.055737,7.746232,18.535434,16,25900
2590,16-25910,0.159867,0.901206,33.916354,0.086245,0.071434,0.002189,0.048867,9.289990,12.765587,16,25910
2803,17-28040,0.063255,3.938648,34.460663,0.014871,0.030711,0.017673,0.006566,9.077927,62.973460,17,28040
2816,18-28170,0.068118,2.728097,28.750154,0.038267,0.014554,0.015297,0.028172,7.944113,112.830093,18,28170
2941,18-29420,0.059822,2.411133,28.897647,0.040331,0.015820,0.003672,0.032801,8.753037,59.336259,18,29420
2976,19-29770,0.084878,7.349896,31.765509,0.031633,0.052523,0.000723,0.001808,5.875670,318.775814,19,29770


### Plots 2: stored data

In [27]:
def f_plot_hist_spec(df,dict_bkg,plot_type):
    
    img_size=128
    assert plot_type in ['hist','spec','spec_relative','grid'],"Invalid mode %s"%(plot_type)

    if plot_type in ['hist','spec','spec_relative']:     fig=plt.figure(figsize=(6,6))
    
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_sdev/np.sqrt(row.num_imgs)
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

        if plot_type=='spec_relative':
            
            y2=row.spec_val
            yerr2=row.spec_sdev
            x2=np.arange(len(y2))
            
            ### Reference spectrum
            y1,yerr1=dict_bkg['spec_val'],dict_bkg['spec_sdev']
            y=y2/(1.0*y1)
            ## Variance is sum of variance of both variables, since they are uncorrelated
            
            # delta_r= |r| * sqrt(delta_a/a)^2 +(\delta_b/b)^2) / \sqrt(N)
            yerr=(np.abs(y))*np.sqrt((yerr1/y1)**2+(yerr2/y2)**2)/np.sqrt(row.num_imgs)

            plt.fill_between(x2, y - yerr, y + yerr, alpha=0.4)
            plt.plot(x2, y, marker=marker, linestyle=':',label=label)
            plt.xlim(0,img_size/2)
            plt.ylim(0.5,2)
            plt.legend()
            
        if plot_type=='grid':
            images=np.load(row.fname)[:,:,:]
            f_plot_grid(images[:18],cols=6,fig_size=(10,5))
            
    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
        plt.xscale('symlog',linthreshx=50)
    
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_sdev']/np.sqrt(num_bkgnd)
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title('Spectrum')
        
    if plot_type=='spec_relative':
        plt.axhline(y=1.0,color='k',linestyle='-.')
    
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    if plot_type in ['hist','spec']:     plt.yscale('log')

# f_plot_hist_spec(df_runs,dict_val,'hist')
# f_plot_hist_spec(df_runs,dict_val,'spec')
# f_plot_hist_spec(df_runs,dict_val,'spec_relative')
# f_plot_hist_spec(df_best,dict_val,'grid')


In [28]:
def f_widget_compare(df,labels_list,bkgnd,plot_type):
    df_temp=df[df.label.isin(labels_list)]
    f_plot_hist_spec(df_temp,dict_val,plot_type)

interact_manual(f_widget_compare,df=fixed(df_best),
                labels_list=SelectMultiple(options=df_best.label.values), 
                bkgnd=fixed(dict_val),plot_type=ToggleButtons(options=['hist','spec','grid','spec_relative']))

interactive(children=(SelectMultiple(description='labels_list', options=('11-17440', '11-18170', '16-25900', '…

<function __main__.f_widget_compare(df, labels_list, bkgnd, plot_type)>

## Plot images

In [29]:
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
s_val=np.load(ip_fname,mmap_mode='r')[:1000,0,:,:]
print(s_val.shape)
f_plot_grid(s_val[100:118],cols=6,fig_size=(6,3))


(1000, 128, 128)
3 6


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …